In [ ]:
%matplotlib inline
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import os

<h1>XGBoost Cloud Prediction Invocation Template</h1>
<h4>Invoke SageMaker Prediction Service</h4>

In [ ]:
import boto3
import re
from sagemaker import get_execution_role
import sagemaker

In [ ]:
# Acquire a realtime endpoint
endpoint_name = 'xgboost-iris-v1'
predictor = sagemaker.predictor.RealTimePredictor(endpoint=endpoint_name)

In [ ]:
from sagemaker.predictor import csv_serializer, json_deserializer

predictor.content_type = 'text/csv'
predictor.serializer = csv_serializer
predictor.deserializer = None

In [ ]:
# Test predictive quality against data in validation file
df_all = pd.read_csv('iris_validation.csv',
                     names=['encoded_class','sepal_length','sepal_width','petal_length','petal_width'])

In [ ]:
df_all.head()

In [ ]:
df_all.columns

In [ ]:
# Need to pass an array to the prediction
# can pass a numpy array or a list of values [[19,1],[20,1]]
# arr_test = df_all.as_matrix(df_all.columns[:-1])
arr_test = df_all.as_matrix(['sepal_length', 'sepal_width', 'petal_length','petal_width'])

In [ ]:
type(arr_test)

In [ ]:
arr_test.shape

In [ ]:
arr_test[:5]

In [ ]:
result = predictor.predict(arr_test[:2])

In [ ]:
arr_test.shape

In [ ]:
result

In [ ]:
int(float('0.0'))

In [ ]:
# For large number of predictions, we can split the input data and
# Query the prediction service.
# array_split is convenient to specify how many splits are needed
predictions = []
for arr in np.array_split(arr_test,10):
    result = predictor.predict(arr)
    result = result.decode("utf-8")
    result = result.split(',')
    print (arr.shape)
    predictions += [int(float(r)) for r in result]

In [ ]:
len(predictions)

In [ ]:
predictions[:5]

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'])

In [ ]:
df_all['class'] = le.inverse_transform(df_all.encoded_class)

In [ ]:
df_all['predicted_class']=le.inverse_transform(predictions)

In [ ]:
df_all.head()

In [ ]:
print('Confusion matrix - Actual versus Predicted')
pd.crosstab(df_all['class'], df_all['predicted_class'])

In [ ]:
import sklearn.metrics as metrics
print(metrics.classification_report(df_all['class'], df_all['predicted_class']))